- карта, котра відображає рівень приросту / скорочення населення за регіонами станом на останній доступний рік

- лінійний графік, котрий відображає рівень приросту / скорочення населення за регіонами за всіма роками (одна лінія на графіку - один регіон)

In [2]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [3]:
df = pd.read_csv('population_trends.csv')
df = df[df.region != 'Ukraine']
df.head()

,region,year,rate
31,Crimea,1989,3.9
32,Crimea,1990,2.5
33,Crimea,1991,0.9
34,Crimea,1992,-0.7
35,Crimea,1993,-2.7


In [4]:
df['year'].max()

2019

In [5]:
ukraine_map = gpd.read_file('ukraine.json')
ukraine_map.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,UKR,Ukraine,UKR.1_1,Cherkasy,Cherkas'ka Oblast'|Cherkasskaya Oblast'|Cherkassy,None,Oblast',Region,None,UA.CK,"MULTIPOLYGON (((31.32614 48.74507, 31.31716 48..."
1,UKR,Ukraine,UKR.2_1,Chernihiv,Chernigov|Tschernigow,None,Oblast',Region,None,UA.CH,"MULTIPOLYGON (((33.09283 50.50966, 33.09261 50..."
2,UKR,Ukraine,UKR.3_1,Chernivtsi,Chernivets'ka Oblast'|Chernovitskaya Oblast'|C...,None,Oblast',Region,None,UA.CV,"MULTIPOLYGON (((24.93280 47.72794, 24.93301 47..."
3,UKR,Ukraine,UKR.4_1,Crimea,Crimée|Criméia|Krim|Krymskaya Respublika|Respu...,None,Autonomous Republic,Autonomous Republic,None,UA.KR,"MULTIPOLYGON (((33.79291 44.39153, 33.79465 44..."
4,UKR,Ukraine,UKR.5_1,Dnipropetrovs'k,Dnipropetrovsk|Dniepropietrovsk|Dnjepropetrowsk,None,Oblast',Region,None,UA.DP,"MULTIPOLYGON (((33.93176 47.48407, 33.92332 47..."


In [6]:
ukraine_map = ukraine_map[['NAME_1', 'geometry']]
ukraine_map = ukraine_map.rename(columns={'NAME_1': 'region'})
ukraine_map.head()

,region,geometry
0,Cherkasy,"MULTIPOLYGON (((31.32614 48.74507, 31.31716 48..."
1,Chernihiv,"MULTIPOLYGON (((33.09283 50.50966, 33.09261 50..."
2,Chernivtsi,"MULTIPOLYGON (((24.93280 47.72794, 24.93301 47..."
3,Crimea,"MULTIPOLYGON (((33.79291 44.39153, 33.79465 44..."
4,Dnipropetrovs'k,"MULTIPOLYGON (((33.93176 47.48407, 33.92332 47..."


In [7]:
df2019 = df.loc[df.year == 2019]
df2019.head()

,region,year,rate
61,Crimea,2019,NaN
92,Vinnytsya,2019,-7.9
123,Volyn,2019,-2.8
154,Dnipropetrovs'k,2019,-8.9
185,Donets'k,2019,NaN


In [8]:
basee = alt.Chart(ukraine_map).mark_geoshape(fill = 'lightgrey', stroke = 'grey').encode(tooltip = [alt.Tooltip('region:N')])
base = basee.properties(width = 800, height = 500)

In [9]:
base.show()

Displaying chart at http://localhost:20947/


In [10]:
line_chart = alt.Chart(df).mark_line().encode(
    x = alt.X('year:Q',
              axis = alt.Axis(format = "1000f")),
    y = alt.Y('rate:Q'),
    detail = alt.Detail('region:N'),
    color=alt.value('grey'),
).properties(title = "Рівень приросту / скорочення населення за регіонами за всіма роками",
             width = 800, height = 600)

In [11]:
line_chart.show()

Displaying chart at http://localhost:20947/


In [12]:
ukraine = alt.Chart(ukraine_map).transform_lookup(
    lookup = 'region',
    from_ = alt.LookupData(data = df2019, 
                           key = 'region',
                           fields=['region', 'rate'])
).mark_geoshape(stroke = 'grey').encode(
    color = alt.Color('rate:Q', 
                      scale = alt.Scale(scheme = 'reds', reverse=True), 
                      title='Rate',
                      legend=alt.Legend(
                            orient = 'bottom-left',
                            gradientLength = 250
                            )),
    tooltip = [
        alt.Tooltip('region:N'),
        alt.Tooltip('rate:N')]
)

rate_map = alt.layer(base, ukraine)
rate_map = rate_map.properties(title = "Рівень приросту / скорочення населення за регіонами станом на 2019 рік",
                               width = 800, height = 500)


In [13]:
rate_map.show()

Displaying chart at http://localhost:20947/


In [14]:
final_chart = alt.hconcat(rate_map.properties(width = 800, height = 500), 
            line_chart.properties(width = 800, height = 500)).properties(background = '#F9F9F9', padding = 25).configure_concat(spacing = 0)

In [15]:
final_chart.show()

Displaying chart at http://localhost:20947/
